El TARGET es el karma y los votos

- Predicción de los sentimientos en los comentarios publicados de Meneame , teniendo en cuenta el karma de la publicación y los votos recividos a la misma.

In [1]:

# Limpieza
import numpy as np
import pandas as pd
import nltk

# Graficas
import matplotlib.pyplot as plt
import seaborn as sns

# Normalizacion
from sklearn.preprocessing import MinMaxScaler

# GridSearchCV
from sklearn.model_selection import GridSearchCV

# Train, Test
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import jaccard_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Validacion
from sklearn.model_selection import StratifiedKFold

In [2]:
df_completo = pd.read_csv("Data/100178_Comentarios.csv", sep=";")
df_completo.shape

(100178, 10)

In [3]:
df_completo.head()

,URL,fecha_envio,fecha_publicacion,pag_comentario,num_comentario,usuario,fecha,texto,votos,karma
0,https://old.meneame.net/story/arresten-lideres...,1715371648,1715379303,1,1,ingenierodepalillos,1715371791,"O que les den de hostias hasta hartarse, son t...",8.0,77.0
1,https://old.meneame.net/story/arresten-lideres...,1715371648,1715379303,1,2,Top_Banana,1715372035,"#1 Sea cual sea la opción, seguro que preferir...",3.0,35.0
2,https://old.meneame.net/story/arresten-lideres...,1715371648,1715379303,1,3,Top_Banana,1715372235,"Un saludo Mysto, ¡hermoso!",4.0,51.0
3,https://old.meneame.net/story/arresten-lideres...,1715371648,1715379303,1,4,CharlesBrowson,1715372556,Es tal es desproposito y la barbarie que no ca...,6.0,52.0
4,https://old.meneame.net/story/arresten-lideres...,1715371648,1715379303,1,5,BigMom,1715372955,"#1 Es el problema, uno no quiere rebajarse. Pe...",3.0,33.0


In [4]:

df_completo.isna().sum()

URL                    0
fecha_envio            0
fecha_publicacion      0
pag_comentario         0
num_comentario         0
usuario                2
fecha                  0
texto                 33
votos                785
karma                785
dtype: int64

In [5]:
df_no_nan = df_completo.dropna()

In [6]:
df_no_nan.isna().sum()

URL                  0
fecha_envio          0
fecha_publicacion    0
pag_comentario       0
num_comentario       0
usuario              0
fecha                0
texto                0
votos                0
karma                0
dtype: int64

In [7]:
df_no_nan.shape

(99358, 10)

In [ ]:
df_no_nan.head()

In [8]:
df_no_nan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99358 entries, 0 to 100177
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   URL                99358 non-null  object 
 1   fecha_envio        99358 non-null  int64  
 2   fecha_publicacion  99358 non-null  int64  
 3   pag_comentario     99358 non-null  int64  
 4   num_comentario     99358 non-null  int64  
 5   usuario            99358 non-null  object 
 6   fecha              99358 non-null  int64  
 7   texto              99358 non-null  object 
 8   votos              99358 non-null  float64
 9   karma              99358 non-null  float64
dtypes: float64(2), int64(5), object(3)
memory usage: 8.3+ MB


In [9]:
df_no_nan.shape

(99358, 10)

In [ ]:
#palabras_especificas = ["www.youtube.","»   ver comentario","»   comentario oculto por penalización","edit","www.",".com",".net",".org",".nl","mas.to","youtu.be","menea.me",".eu","jaja","jeje"]

#mask = df_no_nan["texto"].str.contains("|".join(palabras_especificas), case=False)

#df_filtrado_text = df_no_nan[~mask]

In [ ]:
#df_filtrado_text.shape

In [ ]:
#df_filtrado_text

In [ ]:
#df_texto = df_filtrado_text["texto"]

#df_texto

In [11]:

with open(file = "Data/100178_Comentarios.csv", mode = "r", encoding = "latin-1") as file: 
    texto = file.read()
    
texto = texto.lower()

texto=df_completo["texto"].to_string()

In [14]:
type(texto)

str

In [15]:
tokens = nltk.word_tokenize(text = texto, language = "spanish")


tokens


['0',
 'O',
 'que',
 'les',
 'den',
 'de',
 'hostias',
 'hasta',
 'hartarse',
 ',',
 'son',
 't',
 '...',
 '1',
 '#',
 '1',
 'Sea',
 'cual',
 'sea',
 'la',
 'opción',
 ',',
 'seguro',
 'que',
 'preferir',
 '...',
 '2',
 'Un',
 'saludo',
 'Mysto',
 ',',
 '¡hermoso',
 '!',
 '3',
 'Es',
 'tal',
 'es',
 'desproposito',
 'y',
 'la',
 'barbarie',
 'que',
 'no',
 'ca',
 '...',
 '4',
 '#',
 '1',
 'Es',
 'el',
 'problema',
 ',',
 'uno',
 'no',
 'quiere',
 'rebajarse',
 '.',
 'Pe',
 '...',
 '5',
 '#',
 '4',
 'Bueno',
 ',',
 'el',
 'que',
 'escribe',
 'es',
 'un',
 'periodista',
 'Isra',
 '...',
 '6',
 '#',
 '1',
 'una',
 'soga',
 'y',
 'a',
 'bailar',
 '.',
 'Como',
 'en',
 'los',
 'juicios',
 'de',
 '...',
 '7',
 'No',
 'se',
 'porqué',
 'cuando',
 'oigo',
 'hablar',
 'de',
 'los',
 'sionist',
 '...',
 '8',
 'La',
 'clave',
 'de',
 'bóveda',
 'del',
 'sionismo',
 'es',
 'la',
 'aquiesce',
 '...',
 '9',
 'Israeli',
 'decente',
 '?',
 'Todo',
 'Israeli',
 'no',
 'juzgado',
 'debe',
 '...',
 '10',

In [21]:
len(tokens)

1182855

In [16]:
set(tokens)

{'44092',
 '84665',
 '18841',
 'Blablabla',
 '35461',
 '60316',
 '36155',
 '57109',
 '47273',
 'streissand',
 '27174',
 '35055',
 'tipa',
 '55579',
 '80412',
 '54076',
 'cogiéramos',
 '97335',
 '87179',
 '17096',
 '20054',
 'puesta',
 '72876',
 '41963',
 '52624',
 '27748',
 '82566',
 '44286',
 '88667',
 'pateras',
 '68859',
 'BONUS',
 'pam-pam',
 '42455',
 '50350',
 '¿1000km',
 'camufla',
 'aparatito',
 '70413',
 '98526',
 'bruto',
 'regule',
 'palo',
 '10564',
 '93441',
 '23015',
 '98398',
 '77206',
 '31831',
 '88836',
 '77585',
 '56548',
 '38806',
 '80464',
 '78170',
 '38265',
 '53572',
 '59184',
 '60408',
 'Saldrán',
 '19830',
 '80678',
 '43428',
 '14328',
 '38277',
 'larg',
 'trágame',
 '4221',
 '81772',
 'criticabas',
 '72674',
 '79733',
 '28999',
 '93051',
 '69352',
 '28970',
 'libertarian',
 '33795',
 'Comenta',
 'demonize',
 'genética',
 '16114',
 '16125',
 '20614',
 '71899',
 '41192',
 '210',
 '92932',
 'Inteligenci',
 'ultra-or',
 '19735',
 'inquiocupas',
 'hamas',
 'calzador

In [23]:
print(set(tokens))

{'44092', '84665', '18841', 'Blablabla', '35461', '60316', '36155', '57109', '47273', 'streissand', '27174', '35055', 'tipa', '55579', '80412', '54076', 'cogiéramos', '97335', '87179', '17096', '20054', 'puesta', '72876', '41963', '52624', '27748', '82566', '44286', '88667', 'pateras', '68859', 'BONUS', 'pam-pam', '42455', '50350', '¿1000km', 'camufla', 'aparatito', '70413', '98526', 'bruto', 'regule', 'palo', '10564', '93441', '23015', '98398', '77206', '31831', '88836', '77585', '56548', '38806', '80464', '78170', '38265', '53572', '59184', '60408', 'Saldrán', '19830', '80678', '43428', '14328', '38277', 'larg', 'trágame', '4221', '81772', 'criticabas', '72674', '79733', '28999', '93051', '69352', '28970', 'libertarian', '33795', 'Comenta', 'demonize', 'genética', '16114', '16125', '20614', '71899', '41192', '210', '92932', 'Inteligenci', 'ultra-or', '19735', 'inquiocupas', 'hamas', 'calzadores', 'Defienden', '41169', '89051', '30117', '91209', 'tu', '46342', 'Estéfano', '17022', '80

In [18]:
total_palabras = len(tokens)

print(f"Total de palabras en el texto: {total_palabras}")

palabras_diferentes = len(set(tokens))

print(f"Total de palabras diferentes en el texto: {palabras_diferentes}")

Total de palabras en el texto: 1182855
Total de palabras diferentes en el texto: 159902


In [19]:
riqueza_lexica = palabras_diferentes / total_palabras

print(f"Riqueza Lexica: {riqueza_lexica}")

Riqueza Lexica: 0.1351830951384574


In [25]:
def riqueza_lexica_fun(tokens):
    
    total_palabras = len(tokens)
    palabras_diferentes = len(set(tokens))
    
    riqueza_lexica = palabras_diferentes / total_palabras
    
    return riqueza_lexica

In [26]:
riqueza_lexica_fun(tokens)

0.1351830951384574

In [28]:
texto_nltk = nltk.Text(tokens)

texto_nltk

<Text: 0 O que les den de hostias hasta...>

In [ ]:
plt.figure(figsize = (14,6))
sns.boxplot(x=df_no_nan["votos"])
plt.xticks(rotation=45, ha="right")

In [ ]:
plt.figure(figsize = (14,6))
sns.boxplot(x=df_no_nan["karma"])
plt.xticks(rotation=45, ha="right")